In [32]:
import numpy as np
import pandas as pd
from datetime import datetime
import pandas_profiling

def col_unknown_response_dict():
    """ Return a dictionary of column names as keys and values as a list of unknown/refuse codes to question
    """
    urd = {}
    urd['REGION'] = [9]
    urd['LIVINGQTR'] = [98]
    urd['MARSTAT'] = [0, 99]
    urd['RACEA'] = [900, 970, 980]
    urd['HISPETH'] = [93, 99]
    urd['YRSINUS'] = [0, 7, 8, 9]
    urd['NOWAF'] = [0, 7, 8, 9]
    urd['POORYN'] = [9]
    urd['INCFAM97ON2'] = [97, 98, 99]
    urd['EARNINGS'] = [0, 97, 98, 99]
    urd['USUALPL'] = [0, 7, 8, 9]
    urd['TYPPLSICK'] = [0, 97, 98, 99]
    urd['ALC5UPYR'] = [996, 997, 998, 999]
    urd['SMOKAGEREG'] = [0, 97, 98, 99]
    urd['CIGSDAY'] = [96, 97, 98, 99]
    urd['SMOKESTATUS2'] = [0, 90]
    urd['LANY'] = [22]
    urd['CLIMALC'] = [0, 7, 8, 9]
    urd['HEARING'] = [0, 97, 98, 99]
    urd['AEFFORT'] = [6, 7, 8, 9]
    urd['AHOPELESS'] = [6, 7, 8, 9]
    urd['AWORTHLESS'] = [6, 7, 8, 9]

    return urd


def write_results_header(working_df, working_filename, results_filename):
    """save summary information to results file
    """
    cols_with_nulls = df.columns[df.isnull().any()]
    with open(results_filename, 'w+') as f:   
        f.write('Review of file ' + working_filename + '\n\n')
        f.write('Rows: ' + str(len(working_df.index)) + '\n')
        f.write('Columns with null data: \n')
        if len(cols_with_nulls) == 0:
            f.write('None\n')
        else:
            for col in cols_with_nulls:
                f.write(col + '\n')
        

def count_valid_flag_data(working_df, flag_col_filename, results_filename):
    """create counts of pos/neg responses for flag (yes/no) cols
       append counts to results file
       return list of flag col names
    """
    flag_cols = pd.read_csv(flag_col_filename)
    results_dict = {}

    for col in flag_cols:
        print('checking flag col ', col)
        results_entry = {}

        values = df[col].value_counts().keys().tolist()
        counts = df[col].value_counts().tolist()
        
        print('values: ', values)
        print('counts: ', counts)

        yes_index = values.index(2)
        no_index = values.index(1)
        results_entry['positive'] = counts[yes_index]
        results_entry['negative'] = counts[no_index]
        results_entry['percent_positive'] = round(counts[yes_index] / sum(counts), 2)

        results_dict[col] = results_entry
        
    #sort the results in order of most positives to least positives
    priority_list = sorted(results_dict, key=lambda x: (results_dict[x]['positive']), reverse=True)

    with open(results_filename, 'a') as f:   
        f.write('\nReview of flag data\n')
        f.write('feature, positive_response, negative_response, percent_pos\n')
        for item in priority_list:
            f_str = item + ', ' 
            f_str += str(results_dict[item]['positive']) + ', '
            f_str += str(results_dict[item]['negative']) + ', '
            f_str += str(results_dict[item]['percent_positive']) + '\n'
            f.write(f_str)

    return flag_cols
            

def count_valid_category_data(working_df, results_filename):
    """create counts of unknown/refused/not-used responses for cols with category data
       append counts to results file
    """
    results_dict = {}
    missing_categories = col_unknown_response_dict()
    total_responses = len(working_df.index)
    
    for col in working_df.columns:
        results_entry = {}
        total_missing = 0
        total_responses = 0
        
        #get the codes for missing values for the column
        codes_for_missing = missing_categories.get(col, "") 
        response_codes = working_df[col].value_counts().keys().tolist()

        #if the column has missing codes and if any respones match those codes, create the counts
        if codes_for_missing != "" and set(codes_for_missing).intersection(response_codes): 
            response_code_counts = working_df[col].value_counts().tolist()
            total_responses = sum(response_code_counts)
            for code in response_codes:
                if code in missing_categories[col]:
                    code_index = response_codes.index(code)
                    total_missing += response_code_counts[code_index]
        
            results_entry['valid'] = total_responses - total_missing
            results_entry['missing'] = total_missing
            results_entry['percent_valid'] = round(results_entry['valid'] / total_responses, 2)

            results_dict[col] = results_entry

    
    #sort the results in order of most valid responses to least valid responses
    priority_list = sorted(results_dict, key=lambda x: (results_dict[x]['valid']), reverse=True)
    
    with open(results_filename, 'a') as f:   
        f.write('\nReview of category data\n')
        f.write('feature, valid_response, missing_response, percent_valid\n')
        for item in priority_list:
            f_str = item + ', ' 
            f_str += str(results_dict[item]['valid']) + ', '
            f_str += str(results_dict[item]['missing']) + ', '
            f_str += str(results_dict[item]['percent_valid']) + '\n'
            f.write(f_str)
 
    

def update_columns(working_df, col_info_filename):
    """ update column names to readable versions, update column types
    """
    info_dict = pd.read_csv(col_info_filename, index_col=0, squeeze=True, header=None).to_dict()
    
    #set numerical columns
    working_df['CIGSDAY'] = pd.to_numeric(working_df['CIGSDAY'])
    working_df['SMOKAGEREG'] = pd.to_numeric(working_df['SMOKAGEREG'])
    working_df['FHSTATPR'] = pd.to_numeric(working_df['FHSTATPR'])
    working_df['MOD10FWK'] = pd.to_numeric(working_df['MOD10FWK'])
    working_df['VIG10FWK'] = pd.to_numeric(working_df['VIG10FWK'])
    working_df['STRONGFWK'] = pd.to_numeric(working_df['STRONGFWK'])
    working_df['YRSINUS'] = pd.to_numeric(working_df['YRSINUS'])
    working_df['FNMEDCT'] = pd.to_numeric(working_df['FNMEDCT'])

    return working_df.rename(columns=info_dict)

    
    
if __name__ == '__main__': 
    t0 = datetime.now()
    print("working!")

    working_file_path = '/Users/alexia/Documents/Springboard/Capstone1/Cap1testing/'
    working_file = working_file_path + 'nhis_00009.csv'
    flag_col_file = working_file_path + 'nhis_flag_cols.csv'
    col_info_file = working_file_path + 'Extract9ColMapping.csv'
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    results_file = working_file_path + 'results' + timestamp + '.csv'

    df = pd.read_csv(working_file, parse_dates = True, index_col = 'YEAR', dtype = 'category')
    
    write_results_header(df, working_file, results_file)
    
    df = update_columns(df, col_info_file)
    print(df.info())
    
    #cols_analyzed = count_valid_flag_data(df, flag_col_file, results_file)
    
    ##remove flag columns, get counts on the rest
    #df_rest = df.drop(cols_analyzed, axis=1)
    #count_valid_category_data(df_rest, results_file)
    
    print("Done with analysis, creating Pandas Profiling Report")
    profile = df.profile_report(title='Pandas Profiling Report')
    profile.to_file(output_file= working_file_path + timestamp + "output.html")


    print("Done!")
    total = datetime.now() - t0
    print("time taken: ", total)
    with open(results_file, 'a') as f:   
        f.write('\nReview time = ' + str(total))




working!
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72831 entries, 2018-01-01 to 2018-01-01
Data columns (total 51 columns):
Region of residence                                                     72831 non-null category
Type of living quarters                                                 72831 non-null category
Sex                                                                     72831 non-null category
Legal marital status                                                    72831 non-null category
Main Racial Background                                                  72831 non-null category
Hispanic ethnicity                                                      72831 non-null category
Number of years spent in the U.S.                                       72831 non-null int64
Above or below poverty threshold                                        72831 non-null category
Total combined family income                                            72831 non-null category
Pers